In [1]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())


True

### Connecting to Groq for LLMs

In [2]:
from langchain_groq import ChatGroq 

llm = ChatGroq(
    model='llama-3.1-70b-versatile', # 70 billion parameters with maximum of 8192 tokens
    temperature=0.5,
    max_tokens=4096, # maximum 8000
    timeout=None,
    max_retries=3,
)

messages = [
    ('system', 'You are a helpful assistant that can answer the questions from the users.'),
    ('human', 'Hello there!')
]

response = llm.invoke(messages)
response

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 52, 'total_tokens': 62, 'completion_time': 0.040073665, 'prompt_time': 0.011983758, 'queue_time': 0.0056710300000000005, 'total_time': 0.052057423}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6fdd06f-173f-4fa5-be8b-0664a6b966c6-0', usage_metadata={'input_tokens': 52, 'output_tokens': 10, 'total_tokens': 62})

In [46]:
import time 

for chunk in llm.stream(messages):
    time.sleep(0.1)
    print(chunk.content, end='')


Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?

### Connecting to Pinecone (Vector Store)

In [3]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    raise ValueError("Please set the PINECONE_API_KEY environment variable.")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

c:\Users\Usrer\Desktop\OnceHuman Chatbot\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
import time

index_name = "oncehuman-vector-store"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [5]:
from langchain_huggingface.embeddings.huggingface_endpoint import HuggingFaceEndpointEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding = HuggingFaceEndpointEmbeddings(model='sentence-transformers/all-mpnet-base-v2')
vector_store = PineconeVectorStore(index=index, embedding=embedding)
retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={ 'k': 5, 'top_k': 25, 'lambda_mult': 0.5 })

In [6]:
relevant_docs = retriever.invoke('What is a solar drill?')
print(relevant_docs[0].page_content)

This article is about **Solar Drill** the Tool, did you mean:Solar Drill (Memetic Specialization)?
> An upgraded Electric Rock Drill that offers greatly increased Mining efficiency. It runs entirely on solar power.
Solar Drill
-----------
[![Solar Drill](https://static.wikia.nocookie.net/once-human/images/0/09/Solar_Drill.png/revision/latest?cb=20240917123207)](https://static.wikia.nocookie.net/once-human/images/0/09/Solar_Drill.png/revision/latest?cb=20240917123207 "Solar Drill")
### Rarity
[Legendary](/wiki/Category:Legendary "Category:Legendary")
### Type
[Tools](/wiki/Category:Tools "Category:Tools")
### Weight
1\.000
### Durability
50
How To Get
----------
**Memetic Specialization**:Solar Drill (Memetic Specialization)**Crafted**:[Intermediate Supplies Workbench](/wiki/Intermediate_Supplies_Workbench "Intermediate Supplies Workbench")  
[Advanced Supplies Workbench](/wiki/Advanced_Supplies_Workbench "Advanced Supplies Workbench")  
Contents
--------
* [1 Summary](#Summary)
* [2 Ho

### Creating history-aware retriever and conversational model

In [7]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

HISTORY_AWARE_PROMPT = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

HISTORY_AWARE_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages([
    ('system', HISTORY_AWARE_PROMPT),
    MessagesPlaceholder('chat_history'),
    ('human', '{input}'),
])

history_aware_retriever = create_history_aware_retriever(llm=llm, retriever=retriever, prompt=HISTORY_AWARE_PROMPT_TEMPLATE)

RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\
Just return answer to the question directly. Do not try to make up the next question and answer. \

Context: {context}"""

RAG_SYSTEM_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages([
    ('system', RAG_SYSTEM_PROMPT),
    MessagesPlaceholder('chat_history'),
    ('human', '{input}'),
])

qa_chain = create_stuff_documents_chain(llm=llm, prompt=RAG_SYSTEM_PROMPT_TEMPLATE)
rag_chain = create_retrieval_chain(
    retriever=history_aware_retriever,
    combine_docs_chain=qa_chain,
)


In [13]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer',
)

In [18]:
# limitation: cannot use .stream() method due to version compatibility

import time 

user_prompt = { 'input': 'Can you list down the items and mods required?' }
config = { 'configurable': { 'session_id': '1' } }

conversational_rag_chain.invoke(user_prompt, config=config)


{'input': 'Can you list down the items and mods required?',
 'chat_history': [HumanMessage(content='What is the latest event in Once Human?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know the latest event in Once Human as the provided context only contains information up to October 2024.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is the best build in Once Human?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='There is no single "best" build mentioned in the context, as players have different preferences and builds for different playstyles, such as frost, burn, shock, unstable bomber, and fast gunner.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Okay, how to build unstable bomer build?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The context doesn\'t provide a detailed guide on how to build an unstable bomber, but it mentions a "Quick And Dirty Unstable 